In [ ]:
# # Load the PED file 
# file= "Data/s1.ped"
# import csv
# dict={}
# names=[]
# meta=[]
# # Open the PED file for reading
# with open(file, 'r') as ped_file:
#     # Create a CSV reader object
#     reader = csv.reader(ped_file, delimiter=' ')
#     # For each row in the file   
#     for row in reader:
#         # for each pair of alleles in the row
#         counter=0
#         meta.append(row[0:6])
#         for i in range(6, len(row), 2):
#             # get the two alleles
#             allele1 = row[i]
#             allele2 = row[i+1]
#             counter+=1
#             name= 'var'+str(counter)
            
#             # if the name is not in the dictionary, add it
#             if name not in dict:
#                 dict[name] = str(allele1)+str(allele2)
                
#             # if the name is in the dictionary, add the alleles to the existing string
#             else:
#                 dict[name] += str(allele1)+str(allele2)
# freq={}

# import collections
# for key in dict:
#     freq[key]=collections.Counter(dict[key])
    
# # convert the dictionary to a dataframe
# import pandas as pd
# df = pd.DataFrame.from_dict(freq, orient='index')
# # df[total] sum of the counts of each allele if the allele is present in the population
# df['total']=df.sum(axis=1)

# # each allele is divided by the total to get the frequency of the allele
# for i in range(0, len(df.columns)-1):
#     df.iloc[:,i]=df.iloc[:,i]/df['total']
    
# df.to_csv('Data/s1_freq.csv', sep='\t')
    

In [ ]:
import collections
import pandas as pd

def compute_allele_freqs(file_path):
    # Initialize variables
    dict_alleles = {}
    names = []
    meta = []
    
    # Open the PED file for reading
    with open(file_path, 'r') as ped_file:
        # For each row in the file   
        for row in ped_file:
            row_data = row.strip().split()
            
            # Collect metadata information
            meta.append(row_data[0:6])
            
            # Process allele pairs
            for i in range(6, len(row_data), 2):
                allele1 = row_data[i]
                allele2 = row_data[i+1]
                
                # Create allele name
                name = f'var{i//2}'
                
                # Append alleles to dictionary
                if name not in dict_alleles:
                    dict_alleles[name] = str(allele1) + str(allele2)
                else:
                    dict_alleles[name] += str(allele1) + str(allele2)
    
    # Compute allele frequencies
    freq_dict = {}
    for key in dict_alleles:
        freq_dict[key] = collections.Counter(dict_alleles[key])
        
    # Create pandas DataFrame
    df = pd.DataFrame.from_dict(freq_dict, orient='index')
    
    # Add total column
    df['total'] = df.sum(axis=1)
    
    # Normalize allele frequencies
    for i in range(len(df.columns) - 1):
        df.iloc[:, i] = df.iloc[:, i] / df['total']
        
    return df


In [ ]:
file_path = 'Data/s1.ped'
df = compute_allele_freqs(file_path)
df


In [ ]:
df.to_csv('Data/s1_freq.csv', sep='\t')

# Bim File

variant identifier and minor allele

In [3]:
import pandas as pd
df_bim=pd.read_csv('Data/DataS1.bim', sep='\t', header=None)
df_map= pd.read_csv('Data/s1.map', sep='\t', header=None)
df_bim #iloc[, [1,4]]
df_map #iloc[, [1]]
# for every row if df_bim.iloc[row,1] is not df_map.iloc[:,1] drop the row
df_bim=df_bim[df_bim.iloc[:,1].isin(df_map.iloc[:,1])]
df_bim.columns=['chromosome', 'variantID', 'genetic_distance', 'position', 'minor', 'major']
df_bim

,chromosome,variantID,genetic_distance,position,minor,major
0,1,rs3094315,0.020130,752566,3,1
1,1,rs6696609,0.024457,903426,4,2
2,1,rs9442372,0.026288,1018704,1,3
3,1,rs4970405,0.026674,1048955,3,1
4,1,rs11807848,0.026711,1061166,2,4
...,...,...,...,...,...,...
147223,22,rs6010062,0.740407,51156732,1,3
147224,22,rs715586,0.740465,51163138,4,2
147225,22,rs8137951,0.740478,51165664,1,3
147226,22,rs3810648,0.740745,51175626,3,1


# Ped File

familly id and individual id and allels

## Split the ped file by familly id

In [2]:
import os
 
def split_and_convert(file_path):
    # Create a directory to store output files
    os.makedirs('Data/out', exist_ok=True)
    populations = []
    with open(file_path, 'r') as ped_file:
        for row in ped_file:
            row_data = row.strip().split()
            populations.append(row_data[0])
            pedigree = row_data[0]
            with open(f'Data/out/{pedigree}.txt', 'a') as out_file:
                # Join the pairs of alleles in each row
                new_fields = row_data[:6]
                for i in range(6, len(row_data), 2):
                    new_fields.append(row_data[i] + row_data[i+1])
                # Write the new fields to the output file
                out_file.write(' '.join(new_fields) + '\n')
        
    return list(set(populations))

file_path = 'Data/s1.ped'

populations = split_and_convert(file_path)
print(populations)



['Israel', 'Iran', 'Estonia', 'UnitedKingdom', 'Greece', 'Spain', 'Sweden', 'France', 'Norway', 'Russia', 'Portugal', 'Turkey', 'Serbia', 'CzechRepublic', 'Latvia', 'Jordan', 'Austria', 'Poland', 'Ukraine', 'TheNetherlands', 'Bulgaria', 'Croatia', 'Italy', 'Lithuania', 'Switzerland', 'Denmark', 'Romania', 'Luxembourg', 'Macedonia', 'Georgia', 'Kazakhstan', 'Germany', 'Hungary', 'Armenia']


In [89]:
pop_selection = 'Sweden'
pop_path=f'Data/out/{pop_selection}.txt'
date=1

import pandas as pd
df = pd.read_csv(pop_path, sep=' ', header=None)

# conver df_bim['variantID'] to a list
columns= df_bim['variantID'].to_list()
columns.insert(0, 'familyID')
columns.insert(1, 'individualID')
columns.insert(2, 'fatherID')
columns.insert(3, 'motherID')
columns.insert(4,'Sex')
columns.insert(5, 'date')



df.columns=columns

df


,familyID,individualID,fatherID,motherID,Sex,date,rs3094315,rs6696609,rs9442372,rs4970405,...,rs80055960,rs114003743,rs739365,rs9616945,rs5770820,rs6010062,rs715586,rs8137951,rs3810648,rs78827609
0,Sweden,Saxtorp5164,0,0,2,1,0,0,0,11,...,0,33,22,0,0,0,22,0,0,0
1,Sweden,Olsund,0,0,1,1,11,44,11,11,...,22,33,44,33,11,33,22,33,0,22
2,Sweden,Ajvide70,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Sweden,sf12,0,0,0,1,11,42,33,11,...,22,33,22,33,33,33,22,33,11,22
4,Sweden,RISE179,0,0,1,1,0,0,0,0,...,0,0,0,0,11,0,0,0,0,0
5,Sweden,I0017,0,0,1,1,11,22,0,11,...,22,33,22,33,33,33,22,33,11,44
6,Sweden,I0013,0,0,1,1,11,0,33,11,...,22,33,22,33,33,11,22,11,33,0
7,Sweden,I0011,0,0,2,1,0,22,33,11,...,22,33,0,11,33,11,0,33,11,22
8,Sweden,I0015,0,0,1,1,33,44,11,11,...,22,33,44,0,33,0,22,11,11,22
9,Sweden,I0012,0,0,1,1,11,22,33,11,...,22,33,44,33,11,11,44,33,11,0


In [90]:

minor=df_bim['minor'].to_list()
minor.insert(0, 'familyID')
minor.insert(1, 'individualID')
minor.insert(2, 'fatherID')
minor.insert(3, 'motherID')
minor.insert(4,'Sex')
minor.insert(5, 'phenotype')

minor[6]


3

In [57]:
freq={}
for i in range(6, len(df.columns)):
    minor3=minor[i]
    # number of rows in the dataframe
    count=0
    totall=len(df)*2
    for j in range(0, len(df)):
        # count occurrences of the minor allele in column i
        count+= str(df.iloc[j,i]).count(str(minor3))
    freq[df.columns[i]]=count/totall
    

freq


{'rs3094315': 0.045454545454545456,
 'rs6696609': 0.20454545454545456,
 'rs9442372': 0.09090909090909091,
 'rs4970405': 0.0,
 'rs11807848': 0.022727272727272728,
 'rs4970421': 0.0,
 'rs2887286': 0.13636363636363635,
 'rs113791678': 0.045454545454545456,
 'rs12073590': 0.0,
 'rs111856013': 0.0,
 'rs3843249': 0.2727272727272727,
 'rs6603791': 0.29545454545454547,
 'rs6687029': 0.18181818181818182,
 'rs6692959': 0.045454545454545456,
 'rs28508199': 0.2727272727272727,
 'rs28549710': 0.4318181818181818,
 'rs4648747': 0.20454545454545456,
 'rs4648752': 0.0,
 'rs13303344': 0.18181818181818182,
 'rs6701689': 0.13636363636363635,
 'rs1467218': 0.0,
 'rs72907476': 0.0,
 'rs77602667': 0.0,
 'rs436045': 0.22727272727272727,
 'rs2257182': 0.45454545454545453,
 'rs260512': 0.29545454545454547,
 'rs10797417': 0.0,
 'rs2036084': 0.06818181818181818,
 'rs2132595': 0.2727272727272727,
 'rs4648831': 0.045454545454545456,
 'rs2645086': 0.36363636363636365,
 'rs7545940': 0.18181818181818182,
 'rs6665546':

In [80]:
dfeuro=pd.read_excel('Data/Eurasian - Dataset_tims.xlsx', sheet_name='Eurasian')
dfeuro=pd.DataFrame(dfeuro)
dfeuro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4158 entries, 0 to 4157
Data columns (total 39 columns):
 #   Column                                                                                                                                                                                                                                 Non-Null Count  Dtype  
---  ------                                                                                                                                                                                                                                 --------------  -----  
 0   #                                                                                                                                                                                                                                      4158 non-null   int64  
 1   Sample ID                                                                                                   

In [81]:
dfeuro.rename(columns={'Date mean in BP in years before 1950 CE [OxCal mu for a direct radiocarbon date, and average of range for a contextual date]':'date'}, inplace=True)

In [82]:
dfeuro['date'].describe()

date_range= [0,1000,2000,3000,4000,5000,6000,7000,8000,9000,10000]



In [85]:
# if dfeuro['date'] is in the range of date_range, then dfeuro['cat_date'] is the index of the range
dfeuro['cat_date']=pd.cut(dfeuro['date'], date_range, labels=range(0, len(date_range)-1))

In [87]:
dfeuro['cat_date']


0       5
1       5
2       4
3       5
4       5
       ..
4153    9
4154    1
4155    7
4156    9
4157    4
Name: cat_date, Length: 4158, dtype: category
Categories (10, int64): [0 < 1 < 2 < 3 ... 6 < 7 < 8 < 9]

In [83]:
dfeuro['Master ID']

0        I1658
1         1H07
2         1H13
3         1H14
4         2H06
         ...  
4153       ZKO
4154    ZLNR-1
4155    ZLNR-2
4156      ZMOJ
4157    ZO2002
Name: Master ID, Length: 4158, dtype: object

In [91]:
# locate df['individualID'] in dfeuro['Master ID'] and add df['cat_date'] to the df[date]
for i in range(0, len(df)):
    for j in range(0, len(dfeuro)):
        if df['individualID'][i] == dfeuro['Master ID'][j]:
            df['date'][i]=dfeuro['cat_date'][j]

df

/tmp/ipykernel_72494/109727887.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'][i]=dfeuro['cat_date'][j]


,familyID,individualID,fatherID,motherID,Sex,date,rs3094315,rs6696609,rs9442372,rs4970405,...,rs80055960,rs114003743,rs739365,rs9616945,rs5770820,rs6010062,rs715586,rs8137951,rs3810648,rs78827609
0,Sweden,Saxtorp5164,0,0,2,1,0,0,0,11,...,0,33,22,0,0,0,22,0,0,0
1,Sweden,Olsund,0,0,1,1,11,44,11,11,...,22,33,44,33,11,33,22,33,0,22
2,Sweden,Ajvide70,0,0,0,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Sweden,sf12,0,0,0,1,11,42,33,11,...,22,33,22,33,33,33,22,33,11,22
4,Sweden,RISE179,0,0,1,3,0,0,0,0,...,0,0,0,0,11,0,0,0,0,0
5,Sweden,I0017,0,0,1,7,11,22,0,11,...,22,33,22,33,33,33,22,33,11,44
6,Sweden,I0013,0,0,1,7,11,0,33,11,...,22,33,22,33,33,11,22,11,33,0
7,Sweden,I0011,0,0,2,7,0,22,33,11,...,22,33,0,11,33,11,0,33,11,22
8,Sweden,I0015,0,0,1,7,33,44,11,11,...,22,33,44,0,33,0,22,11,11,22
9,Sweden,I0012,0,0,1,7,11,22,33,11,...,22,33,44,33,11,11,44,33,11,0


In [93]:
freq={}
df=df[df['date']==date]
for i in range(6, len(df.columns)):
    # 
        minor3=minor[i]
        # number of rows in the dataframe
        count=0
        totall=len(df)*2
        for j in range(0, len(df)):
            # count occurrences of the minor allele in column i
            count+= str(df.iloc[j,i]).count(str(minor3))
        freq[df.columns[i]]=count/totall
    

freq

{'rs3094315': 0.0,
 'rs6696609': 0.25,
 'rs9442372': 0.16666666666666666,
 'rs4970405': 0.0,
 'rs11807848': 0.08333333333333333,
 'rs4970421': 0.0,
 'rs2887286': 0.16666666666666666,
 'rs113791678': 0.16666666666666666,
 'rs12073590': 0.0,
 'rs111856013': 0.0,
 'rs3843249': 0.16666666666666666,
 'rs6603791': 0.08333333333333333,
 'rs6687029': 0.5,
 'rs6692959': 0.0,
 'rs28508199': 0.16666666666666666,
 'rs28549710': 0.08333333333333333,
 'rs4648747': 0.08333333333333333,
 'rs4648752': 0.0,
 'rs13303344': 0.16666666666666666,
 'rs6701689': 0.16666666666666666,
 'rs1467218': 0.0,
 'rs72907476': 0.0,
 'rs77602667': 0.0,
 'rs436045': 0.3333333333333333,
 'rs2257182': 0.5,
 'rs260512': 0.25,
 'rs10797417': 0.0,
 'rs2036084': 0.08333333333333333,
 'rs2132595': 0.5,
 'rs4648831': 0.0,
 'rs2645086': 0.3333333333333333,
 'rs7545940': 0.16666666666666666,
 'rs6665546': 0.25,
 'rs10910060': 0.0,
 'rs10910061': 0.0,
 'rs3736330': 0.3333333333333333,
 'rs2494599': 0.08333333333333333,
 'rs10797421'